#### Importation des librairies :

In [3]:
import os
import pandas as pd
import numpy as np
import gc
import mlflow
import mlflow.sklearn
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
import os

#### Importation du fichier : 

In [9]:
# Chemin vers le dossier contenant les fichiers extraits
dossier_data = "C:/Users/paulm/Documents/GitHub/Projet7/data"

# Liste des fichiers dans le dossier data
fichiers_data = os.listdir(dossier_data)
print(f"Fichiers dans le dossier data : {fichiers_data}")

# Exemple de chargement d'un fichier CSV
# Supposons que vous ayez un fichier CSV nommé "train.csv" dans le dossier data
chemin_fichier_train = os.path.join(dossier_data, 'application_test.csv')
train_data = pd.read_csv(chemin_fichier_train)

# Afficher les premières lignes du DataFrame pour vérifier que le fichier a été chargé correctement
print(train_data.head())


Fichiers dans le dossier data : ['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'Projet+Mise+en+prod+-+home-credit-default-risk (1).zip', 'sample_submission.csv']
   SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100001         Cash loans           F            N               Y   
1      100005         Cash loans           M            N               Y   
2      100013         Cash loans           M            Y               Y   
3      100028         Cash loans           F            N               Y   
4      100038         Cash loans           M            Y               N   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          135000.0    568800.0      20560.5         450000.0   
1             0           99000.

#### Initialisation de l'environnement MLFlow :

In [10]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.


In [14]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

## Préparation des données et feature engineering :

### Étape 1: Prétraitement des données et sauvegarde des résultats intermédiaires

In [15]:
# Chemin vers le dossier contenant les fichiers extraits
dossier_data = "C:/Users/paulm/Documents/GitHub/Projet7/data"

# Démarrer une nouvelle exécution d'expérience MLflow
mlflow.start_run()

# Fonction pour charger les données
def load_data(file_name):
    return pd.read_csv(os.path.join(dossier_data, file_name))

# Chargement des données d'application_train
df = load_data('application_train.csv')

# Enregistrement de la taille des données
mlflow.log_metric("application_train_rows", df.shape[0])
mlflow.log_metric("application_train_columns", df.shape[1])

# Chargement du fichier bureau.csv
bureau = load_data('bureau.csv')

# Chargement du fichier bureau_balance.csv
bureau_balance = load_data('bureau_balance.csv')

# Chargement du fichier previous_application.csv
previous_applications = load_data('previous_application.csv')

# Chargement du fichier POS_CASH_balance.csv
pos_cash_balance = load_data('POS_CASH_balance.csv')

# Chargement du fichier installments_payments.csv
installments_payments = load_data('installments_payments.csv')

# Chargement du fichier credit_card_balance.csv
credit_card_balance = load_data('credit_card_balance.csv')

# Fonction pour le traitement du bureau et de l'équilibre du bureau
def process_bureau_and_balance(bureau, bureau_balance, num_rows=None, nan_as_category=True):
    bb, bb_cat = one_hot_encoder(bureau_balance, nan_as_category=True)
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category=True)
    bureau_aggregations = {'DAYS_CREDIT': ['min', 'max', 'mean'],
                           'CREDIT_DAY_OVERDUE': ['max', 'mean'],
                           'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
                           'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],
                           'AMT_CREDIT_MAX_OVERDUE': ['mean'],
                           'CNT_CREDIT_PROLONG': ['sum'],
                           'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
                           'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
                           'AMT_CREDIT_SUM_OVERDUE': ['mean'],
                           'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
                           'AMT_ANNUITY': ['max', 'mean', 'sum'],
                           'MONTHS_BALANCE_MIN': ['min'],
                           'MONTHS_BALANCE_MAX': ['max'],
                           'MONTHS_BALANCE_SIZE': ['mean', 'sum']}
    for col in bureau_cat:
        bureau_aggregations[col] = ['mean']
    bureau_agg = bureau.groupby('SK_ID_CURR').agg(bureau_aggregations)
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    del bureau, bb
    gc.collect()
    return bureau_agg

# Fonction pour le traitement des applications précédentes
def process_previous_applications(previous_applications, num_rows=None, nan_as_category=True):
    prev, cat_cols = one_hot_encoder(previous_applications, nan_as_category=True)
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
        'DAYS_FIRST_DRAWING': ['min', 'max', 'mean'],
        'DAYS_FIRST_DUE': ['min', 'max', 'mean'],
        'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
        'DAYS_LAST_DUE': ['min', 'max', 'mean'],
        'DAYS_TERMINATION': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    prev_agg = prev.groupby('SK_ID_CURR').agg(aggregations)
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    del prev
    gc.collect()
    return prev_agg

# Fonction pour le traitement du solde POS-CASH
def process_pos_cash(pos_cash_balance, num_rows=None, nan_as_category=True):
    pos, cat_cols = one_hot_encoder(pos_cash_balance, nan_as_category=True)
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

# Fonction pour le traitement des paiements d'installations
def process_installments_payments(installments_payments, num_rows=None, nan_as_category=True):
    ins, cat_cols = one_hot_encoder(installments_payments, nan_as_category=True)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    ins['DAYS_LATE_PAYMENT'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DAYS_EARLY_PAYMENT'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DAYS_INSTALMENT': ['max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'PAYMENT_PERC': ['mean', 'sum'],
        'PAYMENT_DIFF': ['mean', 'sum'],
        'DAYS_LATE_PAYMENT': ['mean', 'sum', 'max'],
        'DAYS_EARLY_PAYMENT': ['mean', 'sum', 'max']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    del ins
    gc.collect()
    return ins_agg

# Fonction pour le traitement du solde de la carte de crédit
def process_credit_card_balance(credit_card_balance, num_rows=None, nan_as_category=True):
    cc, cat_cols = one_hot_encoder(credit_card_balance, nan_as_category=True)
    cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
    cc_aggregations = {
        'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],
        'AMT_BALANCE': ['min', 'max', 'mean', 'sum'],
        'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_INST_MIN_REGULARITY': ['min', 'max', 'mean', 'sum'],
        'AMT_PAYMENT_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'mean', 'sum'],
        'AMT_RECIVABLE': ['min', 'max', 'mean', 'sum'],
        'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'mean', 'sum']
    }
    for cat in cat_cols:
        cc_aggregations[cat] = ['mean']
    cc_agg = cc.groupby('SK_ID_CURR').agg(cc_aggregations)
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    del cc
    gc.collect()
    return cc_agg

# Fonction pour encoder en one-hot et gérer les valeurs manquantes
def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Appeler les fonctions de traitement
bureau_processed = process_bureau_and_balance(bureau, bureau_balance)
previous_app_processed = process_previous_applications(previous_applications)
pos_cash_processed = process_pos_cash(pos_cash_balance)
installments_processed = process_installments_payments(installments_payments)
credit_card_processed = process_credit_card_balance(credit_card_balance)

# Fusionner tous les dataframes traités avec le dataframe principal (df)
df = df.merge(bureau_processed, on='SK_ID_CURR', how='left')
df = df.merge(previous_app_processed, on='SK_ID_CURR', how='left')
df = df.merge(pos_cash_processed, on='SK_ID_CURR', how='left')
df = df.merge(installments_processed, on='SK_ID_CURR', how='left')
df = df.merge(credit_card_processed, on='SK_ID_CURR', how='left')

# Enregistrer les données traitées dans un fichier CSV
processed_data_file = os.path.join(dossier_data, 'processed_data.csv')
df.to_csv(processed_data_file, index=False)

# Enregistrer les paramètres finaux et les métriques finales dans MLflow
mlflow.log_param("final_dataframe_shape", df.shape)
mlflow.log_metric("final_dataframe_rows", df.shape[0])
mlflow.log_metric("final_dataframe_columns", df.shape[1])

# Terminer l'exécution de l'expérience MLflow
mlflow.end_run()

C:\Users\paulm\AppData\Local\Temp\ipykernel_45536\3570414519.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
C:\Users\paulm\AppData\Local\Temp\ipykernel_45536\3570414519.py:80: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves a

In [5]:
# Chargement des données
chemin_dossier = "C:/Users/paulm/Documents/Projet 7/Projet7withCSV/data/" 
processed_data = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

# Suppression des colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
processed_data.drop(columns=colonnes_a_supprimer, inplace=True)

In [6]:
# Sélectionner uniquement les colonnes numériques
colonnes_numeriques = processed_data.select_dtypes(include=[np.number])

# Identifier les colonnes contenant des valeurs infinies
colonnes_infinies = colonnes_numeriques.columns.to_series()[np.isinf(colonnes_numeriques).any()]

# Affichage des colonnes avec des valeurs infinies
print("Colonnes avec des valeurs infinies :")
print(colonnes_infinies)

# Identifier les lignes contenant des valeurs infinies
lignes_avec_infinis = processed_data.index[np.isinf(colonnes_numeriques).any(axis=1)]

# Affichage des indices des lignes contenant des valeurs infinies
print("\nIndices des lignes avec des valeurs infinies :")
print(lignes_avec_infinis.tolist())

# Suppression des lignes contenant des valeurs infinies
processed_data.drop(lignes_avec_infinis, inplace=True)

# Sélection des colonnes numériques après la suppression des lignes
colonnes_numeriques = processed_data.select_dtypes(include=[np.number])

# Vérification
print("\nReste-t-il des valeurs infinies ?")
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print(reste_infinis)

# Si des valeurs infinies restent, les remplacer par NaN
if reste_infinis:
    processed_data.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("\nLes valeurs infinies ont été remplacées par NaN.")

# Vérification
reste_infinis = np.isinf(colonnes_numeriques).any().any()
print("\nReste-t-il des valeurs infinies après remplacement ?")
print(reste_infinis)

Colonnes avec des valeurs infinies :
PREV_APP_CREDIT_PERC_MAX      PREV_APP_CREDIT_PERC_MAX
PREV_APP_CREDIT_PERC_MEAN    PREV_APP_CREDIT_PERC_MEAN
INSTAL_PAYMENT_PERC_MEAN      INSTAL_PAYMENT_PERC_MEAN
INSTAL_PAYMENT_PERC_SUM        INSTAL_PAYMENT_PERC_SUM
dtype: object

Indices des lignes avec des valeurs infinies :
[5687, 60477, 79077, 89018, 98509, 126768, 128791, 140426, 152087, 167136, 199103, 201086, 236164, 238381, 272829, 277962, 287300, 292852, 305373]

Reste-t-il des valeurs infinies ?
False

Reste-t-il des valeurs infinies après remplacement ?
False


In [26]:
# Encodage des variables catégorielles avec OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(processed_data), columns=processed_data.columns)

# Vérification des colonnes non numériques
non_numeric_cols = df_encoded.select_dtypes(exclude=[np.number]).columns
df_encoded.drop(columns=non_numeric_cols, inplace=True)

# Gestion des valeurs manquantes avec SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(df_encoded.drop(columns=['TARGET'])), columns=df_encoded.drop(columns=['TARGET']).columns)

# Séparation des caractéristiques (X) et de la cible (y)
y = processed_data['TARGET']

# Enregistrement de processed_data_encoded.csv
df_final = pd.concat([X, y], axis=1)
df_final.to_csv('processed_data_encoded.csv', index=False)

print("Le fichier processed_data_encoded.csv a été généré avec succès.")

Le fichier processed_data_encoded.csv a été généré avec succès.


In [25]:
df_final.describe()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,CC_CNT_INSTALMENT_MATURE_CUM_SUM,CC_NAME_CONTRACT_STATUS_Active_MEAN,CC_NAME_CONTRACT_STATUS_Approved_MEAN,CC_NAME_CONTRACT_STATUS_Completed_MEAN,CC_NAME_CONTRACT_STATUS_Demand_MEAN,CC_NAME_CONTRACT_STATUS_Refused_MEAN,CC_NAME_CONTRACT_STATUS_Sent proposal_MEAN,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_nan_MEAN,TARGET
count,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,...,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.000000,307492.0,307492.000000
mean,153745.500000,0.095215,0.341664,0.340106,0.693677,0.417003,1294.548359,1877.254429,5004.105272,323.930524,...,688.117641,936.417123,0.000173,56.557284,0.001381,0.001105,0.077077,4.639356,0.0,0.080727
std,88765.438824,0.293513,0.474296,0.473745,0.460966,0.721217,578.168031,1348.475263,2836.873372,238.661653,...,623.850563,101.015818,0.013374,93.708263,0.063499,0.052388,0.632377,12.833922,0.0,0.272416
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,76872.750000,0.000000,0.000000,0.000000,0.000000,0.000000,904.000000,674.000000,2782.000000,92.000000,...,688.117641,936.417123,0.000000,56.557284,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,153745.500000,0.000000,0.000000,0.000000,1.000000,0.000000,1313.000000,1638.000000,4633.000000,289.000000,...,688.117641,936.417123,0.000173,56.557284,0.001381,0.001105,0.077077,4.639356,0.0,0.000000
75%,230618.250000,0.000000,1.000000,1.000000,1.000000,1.000000,1754.000000,2742.000000,6747.000000,474.000000,...,688.117641,936.417123,0.000173,56.557284,0.001381,0.001105,0.077077,4.639356,0.0,0.000000
max,307491.000000,1.000000,2.000000,1.000000,1.000000,14.000000,2547.000000,5602.000000,13670.000000,1001.000000,...,4924.000000,1003.000000,5.000000,924.000000,15.000000,13.000000,33.000000,251.000000,0.0,1.000000


In [27]:
print(test['TARGET'])

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
307506    0.0
307507    0.0
307508    0.0
307509    1.0
307510    0.0
Name: TARGET, Length: 307511, dtype: float64
